In [2]:
import os
import requests #Used to service API connection
from lxml import html #Used to parse XML
from bs4 import BeautifulSoup #Used to read XML table on webpage
import pandas as pd
from pandas.io.json import json_normalize
import wget

def makeURL(myUrl, op, myKey, sigunguCd, bjdongCd):
    # myUrl = "http://192.168.1.120/index.php?"
    url = myUrl + '/'+ op + "?ServiceKey=" + myKey + "&sigunguCd=" + sigunguCd + '&bjdongCd=' + bjdongCd
    url = url.rstrip('&')
    return url


def autoDownSource(url):
    outPath = '../data/inbound/'
    filename = url.split('/')[-1].replace('.csv', '')
    createFolder(outPath + filename)
    wget.download(url, outPath + filename)
    

def xmlProcess(url):
    response = requests.get(url)
    # Check if page is up
    if response.status_code == 200:
        # Convert webpage to %Data
        Data = BeautifulSoup(response.text, 'lxml-xml')
        result = []
        rows = 0
        columnName = []
        # search Item all item tag
        iterData = Data.find_all('item')
        for item in iterData:
            item_list = []
            # Fill the value in one row
            for tag in item.find_all():
                try:
                    tagname = tag.name
                    if rows == 0:
                        columnName.append(tagname)
                    item_list.append(item.find(tagname).text)
                except Exception as e:
                    print("This row will be ignored. ", item_list)
            if len(item_list) == 0:
                return pd.DataFrame()
            rows = rows + 1
            result.append(item_list)

    finalResult = pd.DataFrame(result)
    finalResult.columns = columnName
    return finalResult

def jsonProcess(url):

    # 정상 여부 확인 (200 정상)
    response = requests.get(url)
    # JSON 데이터 획득
    json = response.json()
    # PandasDataframe변환
    df = json_normalize(json)
    return df

def csvProcess(url):

    # 정상 여부 확인 (200 정상)
    response = requests.get(url)

    df = pd.read_csv(url, encoding="ms949")
    return df

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

dataList = pd.read_csv("https://docs.google.com/spreadsheets/d/1VngqG-m7G8k1587c21MZoheR1Fz3amp1mJtiBvA1Jb0/export?format=csv&gid=0")
print("### The total number of target data is " + str(len(dataList)))

dataList = dataList[   dataList['사이트'].notnull() ]
print(dataList[["사이트"]])
print("### The total number of filtered data is " + str(len(dataList)))


outPath = "../data/outbound/"
folderList = dataList["폴더명"].tolist()
print(folderList)
for i in folderList:
    createFolder(outPath+i)

dataList = dataList.fillna("")
dataList = dataList.reset_index(drop=True)


inputUrl = 'http://apis.data.go.kr/1611000/ArchPmsService'
inputKey = "ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D"

inputFolder = "340_건축인허가 정보"
inputFile = "getApBasisOulnInfo&getApDongOulnInfo&getApFlrOulnInfo&getApHoOulnInfo&getApImprprInfo&getApHdcrMgmRgstInfo&getApDemolExtngMgmRgstInfo&getApTmpBldInfo&getApWclfInfo&getApPklotInfo&getApAtchPklotInfo&getApExposPubuseAreaInfo&getApHoExposPubuseAreaInfo&getApJijiguInfo&getApRoadRgstInfo&getApPlatPlcInfo&getApHsTpInfo"
inputFile = inputFile.split('&')
inputDataType = "xml"
print(inputFile)

codeDF = pd.read_csv('../data/inbound/allCode2.csv', encoding='ms949')
codeDF.dtypes
codeList = codeDF['법정동코드'].astype(str).tolist()
# codeDF['법정동코드'] = codeDF['법정동코드'].astype(str)
# len(codeDF)


for i in inputFile:
    DF = pd.DataFrame()
    for j in codeList:
        sigunguCd = j[:5]
        bjdongCd = j[5:]
        url = makeURL(inputUrl, i, inputKey, sigunguCd, bjdongCd)
        try:
            tempDF = xmlProcess(url)
            DF = pd.concat([DF, tempDF], sort = False)
        except Exception as e:
            print(url)
    resultname = outPath + inputFolder + '/' + i + '.csv'
    print(resultname)
    DF.to_csv(resultname, encoding = 'ms949', index = False)

### The total number of target data is 422
                                                   사이트
305  https://www.data.go.kr/dataset/fileDownload.do...
306  http://openapi.kab.co.kr/OpenAPI_ToolInstallPa...
307  http://apis.data.go.kr/1611000/nsdi/LandCharac...
308  http://apis.data.go.kr/1611000/nsdi/LandCharac...
309  http://apis.data.go.kr/1611000/nsdi/BuildingAg...
310  http://apis.data.go.kr/1611000/nsdi/BuildingAg...
311  http://apis.data.go.kr/1611000/nsdi/Fluctuatio...
312  http://apis.data.go.kr/1611000/nsdi/BuildingUs...
313  http://apis.data.go.kr/1611000/nsdi/BuildingUs...
318         https://data.myhome.go.kr/rentalHouseList?
323  http://openapi.kab.co.kr/OpenAPI_ToolInstallPa...
324  http://openapi.kab.co.kr/OpenAPI_ToolInstallPa...
345      http://apis.data.go.kr/1611000/ArchPmsService
347        http://fsc.go.kr/downManager?bbsid=BBS0069&
350  http://apis.data.go.kr/1611000/ArchPmsService/...
354  http://data.insight.go.kr/openapi/service/Pric...
357  http://openapi.mo

http://apis.data.go.kr/1611000/ArchPmsService/getApHoOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=11000&bjdongCd=00000
http://apis.data.go.kr/1611000/ArchPmsService/getApHoOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=11110&bjdongCd=00000
http://apis.data.go.kr/1611000/ArchPmsService/getApHoOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=11110&bjdongCd=10100
http://apis.data.go.kr/1611000/ArchPmsService/getApHoOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=11110&bjdongCd=10200
http://apis.data.go.kr/1611000/ArchPmsService/getApHoOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=11110&

KeyboardInterrupt: 

In [10]:
autoDownSource('https://github.com/joes5370/BJCode/raw/master/allCode.csv')

100% [......................................................] 2085718 / 2085718

In [9]:
def autoDownSource(url):
    outPath = '../data/inbound/'
    filename = url.split('/')[-1].replace('.csv', '')
    createFolder(outPath + filename)
    wget.download(url, outPath + filename)

In [5]:
import os
import requests #Used to service API connection
from lxml import html #Used to parse XML
from bs4 import BeautifulSoup #Used to read XML table on webpage
import pandas as pd
from pandas.io.json import json_normalize
import wget
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
